In [42]:
import os
import jieba

pos_data = []
neg_data = []

# 读取数据
for file_name in os.listdir('正面/'):
    with open('正面/' + file_name, 'r', encoding='utf-8') as in_data:
        line = in_data.readlines()[0].strip()
        line = ' '.join(jieba.cut(line))
        pos_data.append(line)

for file_name in os.listdir('负面/'):
    with open('负面/' + file_name, 'r', encoding='utf-8') as in_data:
        line = in_data.readlines()[0].strip()
        line = ' '.join(jieba.cut(line))
        neg_data.append(line)

In [43]:
# 停用词
stop_words = []
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        stop_words.append(line.strip())

In [45]:
# 删除停用词
pos_data_del_stopwords = []
for item in pos_data:
    words = [w for w in item.split() if w not in stop_words]
    pos_data_del_stopwords.append(' '.join(words))
    
neg_data_del_stopwords = []
for item in neg_data:
    words = [w for w in item.split() if w not in stop_words]
    neg_data_del_stopwords.append(' '.join(words))

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

text_data = neg_data_del_stopwords + pos_data_del_stopwords

vector = TfidfVectorizer()
features = vector.fit_transform(text_data).toarray()

labels = np.vstack((np.zeros((len(neg_data_del_stopwords), 1)), np.ones((len(pos_data_del_stopwords), 1))))

data = np.hstack((features, labels))
np.random.shuffle(data)

features = data[:, :-1]
labels = data[:, -1]

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=2)

lr = LogisticRegression().fit(x_train, y_train)
y_pred = lr.predict(x_test)

print(accuracy_score(y_pred, y_test))

0.855


c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [36]:
import os
import sys
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Input,Flatten
from keras.layers import Conv1D,MaxPooling1D,Embedding
from keras.models import Model
from keras.utils.np_utils import to_categorical

TEXT_DATA_DIR='glove.6B.100d.txt'
MAX_SEQUENCE_LENGTH=1000
MAX_NB_WORDS=10000
EMBEDDING_DIM=100
VALIDATION_SPLIT=0.2

# 建立word与向量的索引
embeddings_index={}
f=open(TEXT_DATA_DIR,'r',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

# 选定的类别
selected_categories = [
    'comp.graphics',
    'rec.motorcycles',
    'rec.sport.baseball',
    'misc.forsale',
    'sci.electronics',
    'sci.med',
    'talk.politics.guns',
    'talk.religion.misc']

# 加载数据
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=selected_categories,
                                      remove=('headers', 'footers', 'quotes'))

newsgroups_test = fetch_20newsgroups(subset='test',
                                     categories=selected_categories,
                                     remove=('headers', 'footers', 'quotes'))
texts = newsgroups_train['data']
labels = newsgroups_train['target']
print(len(texts))
print(np.unique(labels))
print(labels)
texts = [t for t in texts]
print(type(texts[0]),texts)

# 文本向量化
tokenizer=Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

word_index=tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_NB_WORDS)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print('Data validation split.')

# 将data划分为训练集和验证集
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]
num_validation_samples=int(VALIDATION_SPLIT*data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


# 创建词嵌入
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # 如果单词没有在word index,全部设置为0.
        embedding_matrix[i] = embedding_vector
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_NB_WORDS,
                            trainable=False)

print("正在初始化模型...")

# 初始化卷积层和池化层
sequence_input=Input(shape=(MAX_NB_WORDS,),dtype='int32')
embedded_sequences=embedding_layer(sequence_input)
x=Conv1D(128,5,activation='relu')(embedded_sequences)
x=MaxPooling1D(5)(x)
x=Conv1D(128,5,activation='relu')(x)
x=MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(8, activation='softmax')(x)

model=Model(sequence_input,preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
print("正在训练模型...")
# 训练模型
model.fit(x_train,y_train,
          batch_size=128,
          nb_epoch=10,
          validation_data=(x_val,y_val))

Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [40]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb


class CNNLSTM():
    def __init__(self):
        pass
    # Embedding
    max_features = 10000
    maxlen = 1000
    embedding_size = 128

    # Convolution
    kernel_size = 5
    filters = 64
    pool_size = 4

    # LSTM
    lstm_output_size = 70

    # Training
    batch_size = 30
    epochs = 2

    '''
    Note:
    batch_size is highly sensitive.
    Only 2 epochs are needed as the dataset is very small.
    '''
    def initialize(self):
        print('Build model...')
        model = Sequential()
        model.add(Embedding(self.max_features, self.embedding_size, input_length=self.maxlen))
        model.add(Dropout(0.25))
        model.add(Conv1D(self.filters,
                         self.kernel_size,
                         border_mode='valid',
                         activation='relu'))
        model.add(MaxPooling1D(pool_length=self.pool_size))
        model.add(LSTM(self.lstm_output_size))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        print('Model Compiled')
        return model

    def train(self, model):
        print('Loading data...')
        (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=self.max_features)
        print(len(x_train), 'train sequences')
        print(len(x_test), 'test sequences')

        print('Pad sequences (samples x time)')
        x_train = sequence.pad_sequences(x_train, maxlen=self.maxlen)
        x_test = sequence.pad_sequences(x_test, maxlen=self.maxlen)
        print('x_train shape:', x_train.shape)
        print('x_test shape:', x_test.shape)


        print('Train...')
        model.fit(x_train, y_train,
                  batch_size=self.batch_size,
                  epochs=self.epochs,
                  validation_data=(x_test, y_test))
        score, acc = model.evaluate(x_test, y_test, batch_size=self.batch_size)
        print('Test score:', score)
        print('Test accuracy:', acc)

model = CNNLSTM().initialize()

# ======

# from __future__ import print_function

import os
import sys
import numpy as np
from pprint import pprint

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.utils.np_utils import to_categorical

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

# from cnn_lstm import CNNLSTM
from sklearn.externals import joblib

selected_categories = [
    'comp.graphics',
    'comp.windows.x',
    'rec.motorcycles',
    'rec.sport.baseball',
    'sci.crypt',
    'sci.med',
    'talk.politics.guns',
    'talk.religion.misc']

newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=selected_categories,
                                      remove=('headers', 'footers', 'quotes'))

newsgroups_test = fetch_20newsgroups(subset='test',
                                     categories=selected_categories,
                                     remove=('headers', 'footers', 'quotes'))
texts = newsgroups_train['data']
labels = newsgroups_train['target']

print(len(texts))
print(np.unique(labels))
print(labels)

texts = [t for t in texts]
print(type(texts[0]))

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


ax_features = 10000
maxlen = 1000
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]
#
print('Initialize model.')

model = CNNLSTM().initialize()

print('x_train shape:', x_train.shape)
print('x_test shape:', x_val.shape)


print('Train...')
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=epochs,
          validation_data=(x_val, y_val), verbose=1)
score, acc = model.evaluate(x_val, y_val, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets

logreg = linear_model.LogisticRegression(C=1e5, verbose=1)

# we create an instance of Neighbours Classifier and fit the data.
print('Training')

logreg.fit(x_train, y_train)
joblib.dump(logreg, 'logreg.pkl')


clf = joblib.load('logreg.pkl')

pred = clf.predict(x_train)
print(((pred == y_train).sum() * 100.0) / (np.shape(y_train)[0] * 1.0))


# from sklearn import datasets
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# gnb.fit(x_train, y_train)
# joblib.dump(gnb, 'gnb.pkl')
# clf = joblib.load('gnb.pkl')
# pred = clf.predict(x_train)
# print(((pred == y_train).sum() * 100.0) / (np.shape(y_train)[0] * 1.0))

Build model...


c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, activation="relu", padding="valid")`
c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=4)`


Model Compiled
4484
[0 1 2 3 4 5 6 7]
[3 3 1 ... 3 0 4]
<class 'str'>


c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\keras_preprocessing\text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 48098 unique tokens.
Shape of data tensor: (4484, 1000)
Shape of label tensor: (4484,)
Initialize model.
Build model...
Model Compiled
x_train shape: (3588, 1000)
x_test shape: (896, 1000)
Train...


c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:189: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3588 samples, validate on 896 samples
Epoch 1/2
3588/3588 [==============================] - 99s 27ms/step - loss: -28.4770 - acc: 0.1318 - val_loss: -37.1159 - val_acc: 0.1339
Epoch 2/2
896/896 [==============================] - 5s 6ms/step
Test score: -37.1158660863127
Test accuracy: 0.13392857501250027
Training
[LibLinear]

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


42.196209587513934


c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# 预处理
1、处理英文缩写

2、转换为小写字母

3、删除标点等非英文字符

4、分词

5、词性还原

6、删除停用词

7、拼写检查

In [23]:
from nltk.tokenize import word_tokenize
import re
from nltk.stem.porter import PorterStemmer

t = ['a', 'am', 'i', 'are', 'you']  # 停用词

s = "I'm a student. I don't know. I willn't learing. 14 years ago? Where are you from? looking! eggs?"

s = s.replace("I'm", 'I am').replace("don't", 'do not').replace("willn't", 'will not')  # 处理英文缩写

words_sys = re.sub("[^a-zA-Z]", " ", s.lower())  # 删除标点等非英文字符，转换成小写字母

words_sys = ' '.join(words_sys.split())

words = word_tokenize(words_sys)  # 分词

porter_stem = PorterStemmer()

words = [porter_stem.stem(w) for w in words if w not in t]  # 词性还原，删除停用词
words

['student',
 'do',
 'not',
 'know',
 'will',
 'not',
 'lear',
 'year',
 'ago',
 'where',
 'from',
 'look',
 'egg']